ENVIRONMENT SETTING

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import pandas as pd

In [3]:
import time
import urllib
from tqdm import tqdm, trange

In [4]:
tqdm.pandas()

In [5]:
options = webdriver.ChromeOptions()
# options.add_argument('no-sandbox')
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument('disable-gpu')

In [6]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)

GET LINK

In [7]:
# NeurIPS 2019는 딱히 데이터가 느낌있게 나와주지 않음.

conference_links = [
    'https://openreview.net/group?id=ICLR.cc/2023/Conference#submitted',                            # ICLR 2023
    'https://openreview.net/group?id=ICLR.cc/2022/Conference#submitted-submissions',                # ICLR 2022
    'https://openreview.net/group?id=ICLR.cc/2021/Conference#withdrawn-rejected-submissions',       # ICLR 2021
    'https://openreview.net/group?id=ICLR.cc/2020/Conference#withdrawn-rejected-submissions',       # ICLR 2020
    'https://openreview.net/group?id=ICLR.cc/2019/Conference#withdrawn-rejected-submissions',       # ICLR 2019
    'https://openreview.net/group?id=NeurIPS.cc/2022/Conference#rejected-papers-opted-in-public',   # NeurIPS 2022
    'https://openreview.net/group?id=NeurIPS.cc/2021/Conference#rejected-papers-opted-in-public',   # NeurIPS 2021

]

conference_names = [
    'ICLR_2023',
    'ICLR_2022',
    'ICLR_2021',
    'ICLR_2020',
    'ICLR_2019',
    'NeurIPS_2022',
    'NeurIPS_2021',
]

In [8]:
for name, link in zip(conference_names, conference_links):
    print(f'Crawling from {name}')
    df = pd.DataFrame(columns = ['title', 'link'])

    driver.get(link)
    while len(driver.find_elements(By.CSS_SELECTOR, 'li.note h4 a')) == 0:
        time.sleep(1)

    try:
        num_pages = int([button for button in driver.find_elements(By.CSS_SELECTOR, 'nav > ul.pagination > li') if button.text == '»'][0].get_attribute('data-page-number')) - 1
    except:
        num_pages = 1

    for page in trange(num_pages):
        # Get New Paper Lists
        new_papers = pd.DataFrame(data = [{'title': x.text, 'link': x.get_attribute('href')} for x in driver.find_elements(By.CSS_SELECTOR, 'li.note h4 a') if x.text != ''])
        df = pd.concat([df, new_papers]).reset_index(drop = True)

        # Click Next Button
        if num_pages != 1:
            [button for button in driver.find_elements(By.CSS_SELECTOR, 'nav > ul.pagination > li > a') if button.text == '›'][0].click()

        # Sleep
        time.sleep(10)

    df.to_csv(f'./data/conferences/{name}.csv')

Crawling from ICLR_2023


100%|██████████| 89/89 [17:29<00:00, 11.80s/it]


Crawling from ICLR_2022


100%|██████████| 30/30 [06:33<00:00, 13.11s/it]


Crawling from ICLR_2021


...

100%|██████████| 1/1 [00:32<00:00, 32.38s/it]


Crawling from NeurIPS_2022


100%|██████████| 3/3 [00:35<00:00, 11.74s/it]


Crawling from NeurIPS_2021


100%|██████████| 5/5 [00:57<00:00, 11.59s/it]
